<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/ColabDriveFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%writefile ColabDriveFiles.py
# created: 12/05/2020
# updated: 08/14/2022
from __future__ import absolute_import
from pathlib import Path
from google.colab import drive
from subprocess import check_output, CalledProcessError, STDOUT
import concurrent.futures, glob, json, pip, os, shutil, sys, tarfile
import pkg_resources

import json, os, time, pip, shutil, sys
from time import perf_counter, sleep

from os.path import *
import glob
from pathlib import Path

contentPath=os.getcwd()

bullshitPath = join(contentPath,'sample_data')
if exists(bullshitPath):
    shutil.rmtree(bullshitPath)
    sleep(1)
    
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/PythonFiles')
print(f'cwd: {Path.cwd()}')
# %ls -l
os.chdir(contentPath)

impPath = '/content/drive/MyDrive/BashColors.py'
if not exists('BashColors.py'):
    print(contentPath)
    print(impPath)
    shutil.copy2(impPath, contentPath)
    sleep(1)
    from BashColors import C
else:
    print(f'{basename(impPath)} exists')
    from BashColors import C
    
class ColabDriveFiles(object):
    '''List all files in Collab '''
    def __init__(self):
        self.cdf=ColabDriveFiles
        self.__all__ = self.getMethodList()
        self.contentPath=os.getcwd()
        self.drivePth='/content/drive'
        self.myDrivePth='/content/drive/My Drive'
        self.pythonFilesPth='/content/drive/My Drive/PythonFiles'

        self.allDirectoriesDict={}
        self.allFilesDict={}
        self.allFiles=self.updateFileList().copy()

        print(f'{C.Green}ColabDriveFiles{C.ColorOff}')
        
        self.initialGlobPth=join(self.contentPath,'initialGlobList.json')
        self.initialGlobList=glob.glob('**', recursive=True)

        if not exists(self.initialGlobPth):
            os.chdir(contentPath)
            # self.initialGlobList=glob('**', recursive=True)
            with open("initialGlobList.json", 'w') as f:
                self.initialGlobList = json.load(f)
                

        bs="""
        self.planetsPath = join(self.contentPath, 'planets')

        self.generatorPath = join(self.contentPath, 'DataGenerator')

        self.testPath = join(self.contentPath, 'images')

        if not exists(self.planetsPath):

            self.extractTarfile('OriginalPlanets.tar.gz', silent=False)

        if not exists(self.generatorPath):

            self.extractTarfile('DataGenerator5.tar.gz', silent=False)

        if not exists(self.testPath):

            self.extractTarfile('images.tar.gz', silent=False)

            

            # self.extractTarfile('images.tar.gz'), silent=False)

        """

        super(object, self).__init__()

    def __iter__(self):

        return self

    def __len__(self):

        return len(self.name)

    def __str__(self):

        return "%s(%r)" % (self.__class__, self.__dict__)

    
    def printTime(self, strt, fin = None):
        '''print execution times '''
        if fin ==  None:
            fin = perf_counter()
        totalTime = fin - strt
        totalTime =  round(totalTime, 0)
        if totalTime < 60.0:
            print(f'completed in: {C.BIPurple}{totalTime} second(s){C.ColorOff}')
        elif totalTime >=  60.0:
            minutes = totalTime//60
            seconds = totalTime - (minutes * 60)
            seconds = round(seconds, 2)
            print(f'completed in: {C.BIPurple}{minutes} minutes {seconds} second(s){C.ColorOff}')
            
    def checkPackageAvailability(self, packageName:str, silent=True):
        packageName=str(packageName)
        '''checks for package availability returns:boolean True/False'''
        installed_packages = pkg_resources.working_set
        packagesList = sorted(
            ["%s" % i.key for i in installed_packages]
        )
        if not packageName in packagesList:
            if  not silent:
                print(f'{C.BIPurple}{packageName} is not installed{C.ColorOff}')
            return False
        else:
            if not silent:
                print(f'{C.BICyan}{packageName} is installed{C.ColorOff}')
            return True
    
    def silentSystemCall(self, command, silent=True):
        """
        params:
            command: list of strings, `["pip3", "install", "-q", "-U", "pip"]`
            if not silent...
            returns: output, success
        """
        try:
            output = check_output(command, stderr=STDOUT).decode()
            success = True 
        except CalledProcessError as e:
            output = e.output.decode()
            success = False
        if not silent:
            print(command)
            if success:
                print(f'success: {C.BIGreen}{success}{C.ColorOff}\n{output}')
            elif not success:
                print(f'success: {C.BIRed}{success}{C.ColorOff}\n{output}')
            return output, success
        
    def systemCall(self, command):
        """ 
        params:
            command: list of strings, ex. `["pip3", "install", "-q", "-U", "pip"]`
        returns: output, success
        """
        try:
            output = check_output(command, stderr=STDOUT).decode()
            success = True 
        except CalledProcessError as e:
            output = e.output.decode()
            success = False
        print(command)
        if success:
            print(f'success: {C.BIGreen}{success}{C.ColorOff}\n{output}')
        elif not success:
            print(f'success: {C.BIRed}{success}{C.ColorOff}\n{output}')
        return output, success

    def clearAllCaches(self):
        '''clear all __pycache__ and content'''
        all_files = self.allFiles
        for fil in all_files:
            if isdir(fil) and fil.__contains__('__pycache__'):
                cache_pth=realpath(fil)
                print(f'{C.IRed}{cache_pth}')
                shutil.rmtree(fil)
            else: pass

    def cleanDrive(self):
        '''removes sample_data directory and contents'''
        os.chdir('/content')
        delete_path='/content/sample_data'
        if exists(delete_path):
            shutil.rmtree(delete_path)
            self.updateFileList()
        else: pass

    def updateFileList(self):
        '''Updates self.allFiles'''
        os.chdir('/content')
        file_list=glob.glob('**', recursive=True)
        return file_list

    def listColabFiles(self, silent=True):
        '''List all files available in Collab
        populate self.allDirectoriesDict
        populate self.allFilesDict'''
        dirCount=1
        dirDict={}
        fileCount=0
        fileDict={}
        self.updateFileList()
        if not '/content' in sys.path.copy():
            sys.path.append('/content')
        if dirCount==1 and dirDict=={}:
            dirDict[1]='/content'
        else: pass
        if not silent:
            print(f'{C.Blue}{Path.cwd()}')
        for fil in sorted(self.allFiles):
            fullPath=abspath(fil)
            if isdir(fullPath) and not fullPath.__contains__('__pycache__'):
                if not fullPath in sys.path.copy():
                    sys.path.append(fullPath)
                dirCount+=1
                dirDict[dirCount]=fullPath
                if not silent:
                    print(f'\n{C.Blue}{fullPath}')
            elif isfile(fullPath) and not fullPath.__contains__('__pycache__'):
                fileCount+=1
                fileDict[fileCount]=fullPath
                if not silent:
                    print(f'{C.White}{fileCount}. {basename(fullPath)}')
        self.allDirectoriesDict=dirDict.copy()
        self.allFilesDict=fileDict.copy()

    def printSystemPaths(self):
        '''list system paths'''
        pathCount=0
        sysPaths=sys.path.copy()
        sysPaths=sorted(sysPaths)
        for pth in sysPaths:
            pathCount+=1
            print(f'{pathCount}. {C.Blue}{pth}')

    def getMethodList(self, silent=True):
        '''List all methods in AllInstall\n Print silent = True'''
        method_list=[]
        for attribute in dir(self):
            # Get the attribute value
            attribute_value = getattr(self, attribute)
            # Check that it is callable
            if callable(attribute_value):
                # Filter all dunder (__ prefix) methods
                if attribute.startswith('__') == False:
                    method_list.append(attribute)
        if not silent:
            print(f'{len(method_list)} callable methods in AllInstall')
            for method in method_list:
                print(method)
        return method_list

    def listNewFiles(self, deleteNew = False):
        '''Lists newly created files'''
        initial = self.initialGlobList
        currentFilesGlob = glob.glob('**', recursive = True)
        print(f'{C.BICyan}New files...{C.ColorOff}')
        if len(initial) ==  len(currentFilesGlob):
            print(f'{C.BIRed}No new files.\nNothing to see here')
        for fil in currentFilesGlob:
            if not fil in initial:
                if isdir(fil):
                    print(f'{C.BIBlue}{fil}')
                    if deleteNew:
                        shutil.rmtree(fil)
                elif isfile(fil):
                    print(f'{C.ColorOff}{fil}')
                    if deleteNew:
                        os.remove(fil)

cdf=ColabDriveFiles()

In [ ]:

cdf.listNewFiles()

In [ ]:
cdf.pythonFilesPath
# tpuPath=join(cdf)